In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@dataclass
class SDFTransformerConfig:
    dim_context: int = 4
    dim_input: int = 3
    num_outputs: int = 1
    dim_output: int = 1
    delta: float = 0.1
    dim_hidden: int = 128
    num_ctx_seeds: int = 32
    num_x_seeds: int = 32
    num_heads: int = 4

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.proj_x = nn.Linear(config.dim_input, config.dim_hidden)
        self.proj_ctx = nn.Linear(config.dim_context, config.dim_hidden)
        self.pool_ctx = PMA(config.dim_hidden, config.num_heads, config.num_ctx_seeds)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads)
        self.sab1 = SAB(config.dim_hidden, config.dim_hidden, config.num_heads)
        self.sab2 = SAB(config.dim_hidden, config.dim_hidden, config.num_heads)
        self.pool_final = PMA(config.dim_hidden, config.num_heads, config.num_outputs)
        self.proj_final = nn.Linear(config.dim_hidden, config.dim_output)

        self.silu = nn.SiLU()

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        x = self.proj_x(x)                                  # [B, 1, H]
        x = x.expand(-1, self.config.num_x_seeds, -1)       # [B, X, H]
        y = self.pool_ctx(self.proj_ctx(context))           # [B, Y, H]
        xy = self.cross(x, y)                               # [B, X, H]
        xy = xy + x                                         # [B, X, H]
        xy = self.silu(self.sab1(xy))                       # [B, X, H]
        xy = self.silu(self.sab2(xy))                       # [B, X, H]
        xy = self.pool_final(xy)                            # [B, F, H]
        xy = self.proj_final(xy)                            # [B, F, O]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(xy, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': xy}

config = SDFTransformerConfig()
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2025_01_09_fast_enc'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2025_01_09_fast_enc-2025-01-10-08-40-12


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 64
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.02,   "lambda": 0.0,  'learning_rate': 5e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.0075, "lambda": 0.15, 'learning_rate': 4e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.004,  "lambda": 0.3,  'learning_rate': 3e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.002,  "lambda": 0.4,  'learning_rate': 2e-5, 'resolution': 100},
    {"epochs": 2, "epsilon": 0.0,    "lambda": 0.5,  'learning_rate': 1e-5, 'resolution': 256}
]

In [6]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=stage['resolution'], context_size=256)
train_dataset.close()
val_dataset.close()

  0%|          | 0/23438 [00:00<?, ?it/s]

{'loss': 0.0145, 'grad_norm': 0.2654520571231842, 'learning_rate': 4.997866712176807e-05, 'epoch': 0.0}
{'loss': 0.0046, 'grad_norm': 0.2058347761631012, 'learning_rate': 4.995733424353614e-05, 'epoch': 0.0}
{'loss': 0.0041, 'grad_norm': 0.2048361748456955, 'learning_rate': 4.993600136530421e-05, 'epoch': 0.0}
{'loss': 0.0035, 'grad_norm': 0.08083884418010712, 'learning_rate': 4.991466848707228e-05, 'epoch': 0.0}
{'loss': 0.0038, 'grad_norm': 0.08346028625965118, 'learning_rate': 4.9893335608840345e-05, 'epoch': 0.0}
{'loss': 0.0044, 'grad_norm': 0.06707226485013962, 'learning_rate': 4.9872002730608416e-05, 'epoch': 0.01}
{'loss': 0.0027, 'grad_norm': 0.07606853544712067, 'learning_rate': 4.985066985237649e-05, 'epoch': 0.01}
{'loss': 0.0043, 'grad_norm': 0.15545155107975006, 'learning_rate': 4.982933697414455e-05, 'epoch': 0.01}
{'loss': 0.0029, 'grad_norm': 0.09718246012926102, 'learning_rate': 4.980800409591262e-05, 'epoch': 0.01}
{'loss': 0.0043, 'grad_norm': 0.05675332620739937, '

  0%|          | 0/2344 [00:00<?, ?it/s]

{'eval_loss': 0.0011279998579993844, 'eval_runtime': 92.1198, 'eval_samples_per_second': 1628.314, 'eval_steps_per_second': 25.445, 'epoch': 1.0}
{'loss': 0.0013, 'grad_norm': 0.10925262421369553, 'learning_rate': 2.499786671217681e-05, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.12925025820732117, 'learning_rate': 2.4976533833944876e-05, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.040292464196681976, 'learning_rate': 2.4955200955712947e-05, 'epoch': 1.0}
{'loss': 0.0013, 'grad_norm': 0.17638961970806122, 'learning_rate': 2.4933868077481015e-05, 'epoch': 1.0}
{'loss': 0.0012, 'grad_norm': 0.16396550834178925, 'learning_rate': 2.4912535199249083e-05, 'epoch': 1.0}
{'loss': 0.0013, 'grad_norm': 0.0, 'learning_rate': 2.4891202321017154e-05, 'epoch': 1.0}
{'loss': 0.001, 'grad_norm': 0.05936209857463837, 'learning_rate': 2.4869869442785222e-05, 'epoch': 1.01}
{'loss': 0.0009, 'grad_norm': 0.1940104216337204, 'learning_rate': 2.484853656455329e-05, 'epoch': 1.01}
{'loss': 0.0013, 'grad_

  0%|          | 0/2344 [00:00<?, ?it/s]

...

  0%|          | 0/2344 [00:00<?, ?it/s]

{'eval_loss': 0.0021851726341992617, 'eval_runtime': 98.1791, 'eval_samples_per_second': 1527.821, 'eval_steps_per_second': 23.875, 'epoch': 2.0}
{'train_runtime': 1688.723, 'train_samples_per_second': 888.245, 'train_steps_per_second': 13.879, 'train_loss': 0.0028582986993221653, 'epoch': 2.0}


Processing models:   0%|          | 0/3 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/244141 [00:00<?, ?it/s]

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [160, 32] but got: [256, 32].

In [11]:
generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=256, context_size=256)

Processing models:   0%|          | 0/3 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/262144 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/262144 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/262144 [00:00<?, ?it/s]

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)